In [2]:
import boto3

access_key_id = "minioadmin"
secret_access_key = "minioadmin"
minio_url = "http://localhost:9000"

minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

In [3]:
bucket_new = "persistent-landing"

try:
    minio_client.head_bucket(Bucket=bucket_new)
    print(f"Bucket '{bucket_new}' ja existeix")
except:
    minio_client.create_bucket(Bucket=bucket_new)

Bucket 'persistent-landing' ja existeix


In [1]:
import os
from tqdm import tqdm

bucket_anterior = "landing-zone"
paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_anterior):
    for obj in tqdm(page.get("Contents", []), desc="Reorganitzant fitxers"):
        key = obj["Key"]

        # Detectar tipus de fitxer
        filename = key.strip().lower()  # 👈 neteja espais, minúscules

        if filename.endswith(".mp3"):
            new_key = f"audios/{os.path.basename(key)}"
        elif filename.endswith((".png", ".jpg", ".jpeg")):
            new_key = f"imagenes/{os.path.basename(key)}"
        elif filename.endswith((".csv", ".parquet")):
            new_key = f"tabular/{os.path.basename(key)}"
        elif filename.endswith((".txt", ".md", ".json")):
            print("1")
            new_key = f"textos/{os.path.basename(key)}"
        else:
            new_key = f"otros/{os.path.basename(key)}"
    
        minio_client.copy_object(
            Bucket = bucket_new,
            CopySource={"Bucket": bucket_anterior, "Key": key},
            Key = new_key
        )

NameError: name 'minio_client' is not defined